In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as porterStemmer

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\romneyTweets_Filtered.xlsx");
data.head(5)

,date,time,Anootated tweet,Class
0,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",NaN
1,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1.0
2,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1.0
3,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1.0
4,2012-10-16 00:00:00,10:11:43-05:00,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1.0


In [4]:
annotatedTweet = data['Anootated tweet']
actualClass = data['Class']
lengthObama = len(data);

annotatedTweet = annotatedTweet.drop(0)
actualClass = actualClass.drop(0)


annotatedTweetList = annotatedTweet.tolist();
actualClassList = actualClass.tolist();

In [5]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:,%\?-]+', r' ', processedTweet)
        processedTweet = re.sub(r'[.]+', r'', processedTweet)
        
        #processedTweet = re.sub(r'(\s)#\w+', r'', processedTweet)
        #processedTweet = re.sub(r'#\w+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetList:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

In [63]:
len(finalFilteredTweet)

5648

In [7]:
finalFilteredTweet[:3]

['insidi mitt romney bain help philip morri get US high schooler hook On cigarett via',
 'you mean like romney cheat in primari',
 'mitt romney still doesnt believ that we have a black presid']

In [65]:
trainData, testTrainData , y_train, y_test = train_test_split(finalFilteredTweet, actualClassList, test_size = 0.15, random_state = 40)

In [66]:
len(trainData)
len(testTrainData)

848

In [67]:
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [68]:
train_vectors = vectorizer.fit_transform(trainData);
test_vectors = vectorizer.transform(testTrainData);
total_vectors = vectorizer.fit_transform(finalFilteredTweet);

#knn = KNeighborsClassifier()
classifier_sgd = MultinomialNB()
#classifier_sgd = linear_model.SGDClassifier()

In [26]:
classifier_sgd.fit(train_vectors, y_train)
#knn.fit(train_vectors, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
predicted_labels = classifier_sgd.predict(test_vectors)
#predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
classifier_sgd.score(test_vectors, y_test)

0.57193396226415094

In [28]:
target_classifiers = ['-1', '0', '1']
print(classification_report(y_test, predicted_labelsList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.58      0.90      0.70       425
          0       0.50      0.24      0.32       252
          1       0.63      0.26      0.37       171

avg / total       0.57      0.57      0.52       848



In [80]:
cvScores = cross_val_score(classifier_sgd, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores)

[ 0.54506752  0.49450755  0.48822592  0.53298344  0.52569857  0.5614167
  0.56821684  0.49546279  0.51838427  0.52454748]


In [81]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.52545111057288951)
('Minimum: ', 0.48822592489503619)
('Maximum: ', 0.56821684366125014)


In [31]:
cvPredict = cross_val_predict(classifier_sgd, total_vectors, actualClassList, cv=10)
print (cvPredict)

[ 1. -1. -1. ..., -1. -1. -1.]


In [32]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.59      0.86      0.70      2893
        0.0       0.45      0.24      0.32      1680
        1.0       0.60      0.29      0.39      1075

avg / total       0.55      0.57      0.53      5648



In [79]:
bdt = AdaBoostClassifier(classifier_sgd,
                         algorithm="SAMME",
                         n_estimators=200)

In [49]:
bdt.fit(train_vectors, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [87]:
cvScores = cross_val_score(bdt, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.518846590786


In [70]:
cvPredict = cross_val_predict(bdt, total_vectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  0. -1. ...,  0. -1.  0.]


In [71]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.61      0.69      0.65      2893
        0.0       0.39      0.36      0.38      1680
        1.0       0.50      0.37      0.42      1075

avg / total       0.52      0.53      0.52      5648



In [83]:
clf = RandomForestClassifier(n_estimators=10)

In [88]:
cvScores = cross_val_score(clf, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.517680398984


In [84]:
cvPredict = cross_val_predict(clf, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ..., -1. -1. -1.]


In [85]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.60      0.78      0.68      2893
        0.0       0.41      0.31      0.35      1680
        1.0       0.49      0.30      0.37      1075

avg / total       0.52      0.55      0.52      5648

